In [1]:
import os
import random
import zipfile
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def apply_gaussian_blur(image):
    sigma = random.uniform(0, 0.2)
    image = tf.image.gaussian_blur(image, sigma=(sigma, sigma), padding='SAME')
    contrast_factor = random.uniform(0.2, 1.8)
    image = tf.image.random_contrast(image, lower=contrast_factor, upper=contrast_factor)
    saturation_factor = random.uniform(0.2, 1.8)
    image = tf.image.random_saturation(image, lower=saturation_factor, upper=saturation_factor)
    return image * 255.0  # Adjust the range to [0, 255]

def apply_elastic_deformation(image):
    alpha = random.uniform(0.1, 0.3)
    sigma = random.uniform(0.1, 0.3)
    height, width, _ = image.shape
    displacement_field = np.random.rand(height, width, 2) * alpha * sigma
    displacement_field = displacement_field.astype(np.float32)
    x = np.arange(width)
    y = np.arange(height)
    gx, gy = np.meshgrid(x, y)
    gx = np.clip(gx + displacement_field[:, :, 0], 0, width - 1)
    gy = np.clip(gy + displacement_field[:, :, 1], 0, height - 1)
    gx = tf.cast(gx, dtype=tf.int32)
    gy = tf.cast(gy, dtype=tf.int32)
    indices = tf.stack([gy, gx], axis=-1)
    indices = tf.reshape(indices, (-1, 2))
    deformed_image = tf.gather_nd(image, indices)
    deformed_image = tf.reshape(deformed_image, (height, width, 3))
    return deformed_image

def generate_augmented_images(input_dir, output_dir, target_size=(224, 224), num_augmentations=1):
    image_data_generator = ImageDataGenerator(
        rotation_range=random.randint(0, 360),
        width_shift_range=random.uniform(-200, 200),
        height_shift_range=random.uniform(-200, 200),
        shear_range=random.uniform(-200, 200),
        zoom_range=random.uniform(-200, 200),
        horizontal_flip=random.choice([True, False]),
        vertical_flip=random.choice([True, False]),
        preprocessing_function=apply_gaussian_blur)
    
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            img_path = os.path.join(root, file)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)
            for i in tqdm(range(num_augmentations), desc=f"Processing {file}"):
                augmented_image = apply_elastic_deformation(img_array[0])
                augmented_image = augmented_image / 255.0  # Normalize back to [0, 1]
                
                if num_augmentations == 1:
                    image_name = file
                else:
                    image_name = f"augmented_image_{file.split('_')[1].split('.')[0]}_{i}.jpg"
                
                image_path = os.path.join(output_dir, image_name)
                tf.keras.preprocessing.image.save_img(image_path, augmented_image)

def rename_files(folder_path):
    file_names = os.listdir(folder_path)
    counter = 1
    for file_name in tqdm(file_names, desc="Renaming files"):
        new_file_name = f"augmented_image_{counter:02d}.jpg"
        os.rename(os.path.join(folder_path, file_name), os.path.join(folder_path, new_file_name))
        counter += 1
        
def zip_files(source_folder, output_directory, zip_filename):
    try:
        if not os.path.exists(source_folder):
            raise FileNotFoundError(f"The folder '{source_folder}' does not exist.")
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        files_to_zip = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
        zip_path = os.path.join(output_directory, zip_filename)
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for file in files_to_zip:
                file_path = os.path.join(source_folder, file)
                zipf.write(file_path, arcname=file)
        print(f"Files zipped successfully to '{zip_path}'.")
    except Exception as e:
        print(f"Error zipping files: {e}")
    try:
     files = os.listdir(source_folder)
     for file in files:
       file_path = os.path.join(source_folder, file)
       if os.path.isfile(file_path):
         os.remove(file_path)
     print("All files deleted successfully.")
    except OSError:
     print("Error occurred while deleting files.")
print("Success")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Success


In [2]:
if __name__ == "__main__":
    input_dir = "/kaggle/input/validation-data1"
    output_dir = "/kaggle/working/"
    directory = '/kaggle/working/augmented_data'
    zip_filename = 'augmented_data'
    
    generate_augmented_images(input_dir, output_dir, num_augmentations=1)
    rename_files(output_dir)
    zip_files(output_dir, directory, zip_filename)

Renaming files: 100%|██████████| 2969/2969 [00:00<00:00, 41134.21it/s]


Files zipped successfully to '/kaggle/working/augmented_data/augmented_data'.
All files deleted successfully.


In [3]:
# Delete all the contents from output directory
import os
import shutil
def delete_files_in_directory(directory_path):
    try:
        # Iterate over all files and subdirectories
        for root, dirs, files in os.walk(directory_path):
            for file in files:
                file_path = os.path.join(root, file)
                os.remove(file_path)
            for dir_name in dirs:
                dir_path = os.path.join(root, dir_name)
                shutil.rmtree(dir_path)
        print("All files and directories deleted successfully.")
    except Exception as e:
        print(f"Error occurred while deleting files and directories: {e}")

# Usage
directory_path = '/kaggle/working/'
delete_files_in_directory(directory_path)

All files and directories deleted successfully.
